In [302]:
fontTypes=[cv2.FONT_HERSHEY_SIMPLEX, cv2.FONT_HERSHEY_PLAIN, cv2.FONT_HERSHEY_DUPLEX, cv2.FONT_HERSHEY_COMPLEX, cv2.FONT_HERSHEY_TRIPLEX, cv2.FONT_HERSHEY_COMPLEX_SMALL, cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, cv2.FONT_HERSHEY_SCRIPT_COMPLEX,cv2.FONT_ITALIC]
fontsize=[21,9,21,21,22,15,21,23,23]

In [309]:
import string
rus='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
rus= u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
digit='0123456789'
# charactersList = list(string.ascii_lowercase)+list(string.ascii_uppercase)+list(rus)+list(rus.upper())+list(digit)
# charactersList =['a','b',u'д']
charactersList = list(string.ascii_lowercase)+list(string.ascii_uppercase)+list(digit)

# charactersList
# rus.upper()


In [310]:
from PIL import Image, ImageDraw, ImageFont
import os,glob
import numpy as np
import random
import matplotlib.pylab as plt

import cv2

# charactersList=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']


image = img = np.zeros((512,512,3), np.uint8)+255
h,w,_ = image.shape

for k in range(0,200,20):             
                 colorlist.append((k,k,k))

In [305]:
# colorlist=[(0,0,0)]

In [306]:

# colorlist=[(0,0,0)]
# i=8

# color = (0,0,0)
# fontScale =256//fontsize[i]
# org = (int(h/2)-fontsize[i]*fontScale//3, int(w/2)+fontsize[i]*fontScale//2)
# font = fontTypes[i]
# thickness=5
# text='h'
# image0 = cv2.putText(image, text, org, font, fontScale, 
#                          color, thickness, cv2.LINE_AA, False)
# plt.imshow(image0)


In [311]:
def vertical_enh(image0,k=2):
    new_height=int(image0.shape[0]*k)
    dsize = (image0.shape[1], new_height)
    output = cv2.resize(image0, dsize, interpolation = cv2.INTER_AREA)
    m=new_height-image0.shape[0]
    m=m//2
    output=output[m:new_height-m,:]
#     plt.imshow(output)
    return output
    
def horizontal_enh(image0,k=2):
    new_width=int(image0.shape[1]*k)
    dsize = (new_width,image0.shape[0])
    output = cv2.resize(image0, dsize, interpolation = cv2.INTER_AREA)
    m=new_width-image0.shape[1]
    m=m//2
    output=output[:,m:new_width-m]
#     print(output.shape,new_width)
#     plt.imshow(output)    
    return output
    
# vertical_enh(image0,k=1.2)
# horizontal_enh(image0,k=2.9)

In [312]:
    
for text in charactersList:
    print(f'{text} symbol begins')
    for i in range(9):
        fontScale =256//fontsize[i]
        org = (int(h/2)-fontsize[i]*fontScale//3, int(w/2)+fontsize[i]*fontScale*2//5)
        font = fontTypes[i]
    #     org = (int(h/2), int(w/2))
    #     fontScale = 10
        for color_num,color in enumerate(colorlist):  
            for thickness in range(2,23,4):
                img = np.zeros((512,512,3), np.uint8)+255
                image0 = cv2.putText(img, text, org, font, fontScale, 
                             color, thickness, cv2.LINE_AA, False)

                name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+".jpg"
                isWritten = cv2.imwrite(name, image0)
                k = 1
                while k<1.6:
                    k=k+0.5
                    vert = vertical_enh(image0,k)
                    name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+'_vert'+str(k)+".jpg"
                    isWritten = cv2.imwrite(name, vert)
                    hor = horizontal_enh(image0,k)
                    name="letter_dataset\ "+str(text)+'_font'+str(i)+'_clr'+str(color_num)+'_th'+str(thickness)+'_hor'+str(k)+".jpg"
                    isWritten = cv2.imwrite(name, hor)

9 symbol begins
